# Import Required Libraries

The following code imports the following libraries:

- `mysql.connector.pooling`: A library used to connect to a MySQL database.
- `tqdm`: A library used to display progress bars.
- `sklearn.metrics.pairwise`: A library used to calculate pairwise distances.
- `spacy`: A library used for natural language processing (NLP).
- `pandas`: A library used for data analysis and manipulation.
- `os`: A library used to interact with the operating system.
- `collections`: A library used to work with namedtuples.
- `dotenv`: A library used to load environment variables from a .env file.

Additionally, the code calls the `load_dotenv()` function to load environment variables from a .env file.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import spacy
import os
from collections import namedtuple
from dotenv import load_dotenv
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import gym
import numpy as np
from gym import spaces
import torch.nn as nn
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
import time
load_dotenv()

# Named Tuple Definition and Directory Paths

The `ImageProperties` named tuple contains information about an image, including its name, hex color, tags, make, orientation, width, and height.

The script sets the base folder path for the project as `output_path`. The `images_path`, `metadata_path`, and `config_path` are then set as subdirectories within `output_path`. The `list_of_paths` variable holds all four paths.

In [ ]:
ImageProperties = namedtuple('ImageProperties', ['name', 'hex_color', 'tags', 'make', 'orientation', 'width', 'height'])
# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
config_path = os.path.join(output_path, "config")

list_of_paths = [output_path, images_path, metadata_path, config_path]

Here, we define the preferences of the user

In [ ]:
preferences = {
    'Make': 'Canon',
    'ImageWidth': '301',
    'ImageHeight': '301',
    'Orientation': 1,
    'dominant_color': '#73AD3D',
    'tags': ['vase', 'toilet']
}

# Get Metadata

This function reads the metadata stored in a CSV file and returns the data as a pandas DataFrame.

In [ ]:
def get_metadata():
    """
    Get the metadata from the CSV file and return it as a pandas DataFrame.

    :return: A pandas DataFrame with the metadata
    """
    # Read the CSV file
    df = pd.read_csv(os.path.join(metadata_path, "metadata.csv"))

    return df

## Function to get metadata as ImageProperties

The function `get_metadata_as_imageproperties` retrieves the metadata from the CSV file and converts it into a list of namedtuples, ImageProperties.

The `get_metadata` function is used to get the metadata as a pandas DataFrame.

The required metadata fields are extracted from the DataFrame and stored as lists.

For each row in the DataFrame, a namedtuple ImageProperties is created with the fields 'name', 'hex_color', 'tags', 'make', 'orientation', 'width', and 'height' and appended to the list `image_properties`.

Finally, the list `image_properties` is returned.

In [ ]:
ImageProperties = namedtuple('ImageProperties', ['name', 'hex_color', 'tags', 'make', 'orientation', 'width', 'height'])

def get_metadata_as_imageproperties():
    # Get the metadata from the CSV file
    metadata_df = get_metadata()


    # Create a list of ImageProperties
    image_properties = []

    names = metadata_df['filename'].tolist()
    hex_colors = metadata_df['dominant_color'].tolist()
    tags = metadata_df['tags'].tolist()
    makes = metadata_df['Make'].tolist()
    orientations = metadata_df['Orientation'].tolist()
    widths = metadata_df['ImageWidth'].tolist()
    heights = metadata_df['ImageHeight'].tolist()

    for i in range(len(names)):
        image_properties.append(ImageProperties(names[i], hex_colors[i], ast.literal_eval(tags[i]), makes[i], orientations[i], widths[i], heights[i]))

    return image_properties

## Hex to RGB Conversion Function

The `hex_to_rgb` function is used to convert a hexadecimal color code to its equivalent RGB (Red, Green, Blue) values.

### Parameters
- color (str): A string representing the hexadecimal color code.

### Returns
- A tuple containing the red, green, and blue values of the color as integers.

### Exception
- If there is an error in the conversion process, the function returns a tuple with values (0, 0, 0).


In [ ]:
def hex_to_rgb(color):
    try:
        # remove the # from the color
        color = color[1:]
        # convert the color to rgb values
        rgb = tuple(int(color[i:i + 2], 16) for i in (0, 2, 4))
        return rgb
    except:
        return 0, 0, 0

## Cleaning the Preferences Data

The following code is used to clean the preferences data. It removes any rows with missing dominant color values, converts the dominant color column from a hexadecimal string to a tuple of RGB values, converts the tags column to a list of strings, replaces any NaN values with empty strings, and replaces any 0 values with empty strings.

In [ ]:
def get_clean_preferences(df_preferences):
    # remove the rows with nan in dominant_color
    df_preferences = df_preferences.dropna(subset=['dominant_color'])
    # split dominant color into 4 columns and remove the dominant_color column
    # convert the tags column to a list of strings
    # Replace all NaN values with empty strings with the fillna() method
    df_preferences = df_preferences.fillna(0)
    # convert colors to rgb values
    df_preferences['dominant_color'] = df_preferences['dominant_color'].apply(lambda x: hex_to_rgb(x))
    # replace all 0 values with empty strings
    df_preferences['dominant_color'] = df_preferences['dominant_color'].replace(0, '')

    return df_preferences

# Clean Dataset

The function `get_clean_dataset()` is used to clean the metadata dataset. It performs the following steps:

1. Calls the `get_metadata()` function to retrieve the metadata.
2. Converts the metadata to a pandas DataFrame `df_metadata`.
3. Removes the rows in `df_metadata` that have missing values in the `dominant_color` column.
4. If the `dominant_color` column exists, it splits it into four columns: `color1`, `color2`, `color3`, and `color4`. The `dominant_color` column is then dropped. The new columns are then converted to RGB values.
5. Replaces missing values with zeros and converts the `tags` column to a list of strings.
6. Keeps only the following columns in the DataFrame: `filename`, `Make`, `ImageWidth`, `ImageHeight`, `Orientation`, `DateTimeOriginal`, `tags`, `color1`, `color2`, `color3`, and `color4`.
7. Replaces all zero values with empty strings.
8. Returns the cleaned DataFrame `df_metadata`.

In [ ]:
def get_clean_dataset():
    metadata = get_metadata()
    df_metadata = pd.DataFrame(metadata)
    # remove the rows with nan in dominant_color
    df_metadata = df_metadata.dropna(subset=['dominant_color'])
    # split dominant color into 4 columns and remove the dominant_color column
    if 'dominant_color' in df_metadata.columns:
        df_metadata['color1'] = df_metadata['dominant_color'].apply(lambda x: x[0] if len(x) >= 1 else 0)
        df_metadata['color2'] = df_metadata['dominant_color'].apply(lambda x: x[1] if len(x) >= 2 else 0)
        df_metadata['color3'] = df_metadata['dominant_color'].apply(lambda x: x[2] if len(x) == 3 else 0)
        df_metadata['color4'] = df_metadata['dominant_color'].apply(lambda x: x[3] if len(x) == 4 else 0)
        # convert colors to rgb values
        df_metadata['color1'] = df_metadata['color1'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color2'] = df_metadata['color2'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color3'] = df_metadata['color3'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color4'] = df_metadata['color4'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata = df_metadata.drop('dominant_color', axis=1)
    else:
        df_metadata['color1'] = 0
        df_metadata['color2'] = 0
        df_metadata['color3'] = 0
        df_metadata['color4'] = 0

    # convert the tags column to a list of strings
    df_metadata = df_metadata.fillna(0)
    # remove all columns except filename, tags, color1, color2, color3, color4, Make, Width, Height
    df_metadata = df_metadata[
        ['filename', 'Make', 'ImageWidth', 'ImageHeight', 'Orientation', 'DateTimeOriginal', 'tags', 'color1', 'color2',
         'color3', 'color4']]
    # replace all 0 values with empty strings
    df_metadata['Make'] = df_metadata['Make'].replace(0, '')

    return df_metadata

In [ ]:
df_pref = pd.DataFrame([preferences])
df_preferences = get_clean_preferences(df_pref)
df_preferences.head()

In [ ]:
df_metadata = get_clean_dataset()

In [ ]:
df_metadata.head()

# Color Similarity

# Function to Recommend Similar Colors

This function `recommend_colors` recommends similar colors to a given color based on Euclidean distance. The input color is compared with all the colors in a given dataset and the most similar colors are returned as the output.

The function takes two inputs:
- `df_metadata`: A pandas dataframe that contains the information about the images and their colors.
- `df_preferences`: A pandas dataframe that contains the preferred color.
- `n` (optional): An integer that specifies the number of closest matches to return. If n is not specified, all closest matches are returned.

The function first normalizes the colors in the dataset and the preferred color to be between 0 and 1. Then, the Euclidean distance is calculated between the preferred color and all the colors in the dataset. The dataset is sorted based on the Euclidean distance in ascending order and the top n closest matches are returned as the output.


In [ ]:
def recommend_colors(df_metadata, df_preferences, n=0):
    # Load the dataset into a Pandas DataFrame
    data = df_metadata.copy()

    # Extract the individual r, g, and b values from tupbles in the color columns
    data[['r1', 'g1', 'b1']] = pd.DataFrame(data['color1'].tolist(), index=data.index)
    data[['r2', 'g2', 'b2']] = pd.DataFrame(data['color2'].tolist(), index=data.index)
    data[['r3', 'g3', 'b3']] = pd.DataFrame(data['color3'].tolist(), index=data.index)
    data[['r4', 'g4', 'b4']] = pd.DataFrame(data['color4'].tolist(), index=data.index)

    # Normalize the r, g, and b columns to be between 0 and 1
    data[['r1', 'g1', 'b1', 'r2', 'g2', 'b2', 'r3', 'g3', 'b3', 'r4', 'g4', 'b4']] = data[['r1', 'g1', 'b1', 'r2', 'g2',
                                                                                           'b2', 'r3', 'g3', 'b3', 'r4',
                                                                                           'g4', 'b4']] / 255

    # Normalize the input RGB color to be between 0 and 1
    r, g, b = df_preferences['dominant_color'][0]
    r_norm, g_norm, b_norm = r / 255, g / 255, b / 255

    # Compute the Euclidean distance between the input color and all the colors in the dataset
    data['similarity_dominant_color'] = euclidean_distances(
        [[r_norm, g_norm, b_norm, r_norm, g_norm, b_norm, r_norm, g_norm, b_norm, r_norm, g_norm, b_norm]],
        data[['r1', 'g1', 'b1', 'r2', 'g2', 'b2', 'r3', 'g3', 'b3', 'r4', 'g4', 'b4']])[0]

    # Sort the dataset by Euclidean distance in ascending order and return the top 10 closest matches
    if n == 0:
        closest_matches = data.sort_values('similarity_dominant_color', ascending=True)[
            ['filename', 'color1', 'color2', 'color3', 'color4', 'similarity_dominant_color']]
    else:
        closest_matches = data.sort_values('similarity_dominant_color', ascending=True).head(n)[
            ['filename', 'color1', 'color2', 'color3', 'color4', 'similarity_dominant_color']]

    return closest_matches

In [ ]:
recommend_colors(df_metadata, df_preferences)  # OK

# Tag Similarity

# Purpose
The `recommend_tags` function is used to recommend images in a dataset based on their tags, compared to a set of preferred tags.

# Input
The function takes in three arguments:
- `df_metadata`: a pandas DataFrame containing the metadata for the images in the dataset
- `df_preferences`: a pandas DataFrame containing the preferred tags
- `n`: (optional) the number of recommendations to be returned. The default value is 0, which means that the function will return all recommendations sorted by similarity score.

# Output
The function returns a pandas DataFrame containing the recommended images, sorted by their similarity score to the preferred tags. The DataFrame contains two columns:
- `filename`: the name of the recommended image
- `similarity_tags`: the similarity score between the image's tags and the preferred tags

# Implementation details
The function uses spaCy, a natural language processing library, to compute the similarity between each tag word and each preference word. The similarity score is then computed as the average similarity between the image's tags and the preferred tags. The images are sorted by their similarity score in descending order and the top `n` images are returned. If `n` is not specified, all recommendations will be returned.


In [ ]:
def recommend_tags(df_metadata, df_preferences, n=0, nlp=None):
    # Load the spaCy model if it hasn't been loaded
    if not nlp:
        nlp = spacy.load("en_core_web_md")

    # Define the preferences list and the dataframe
    preferences = df_preferences['tags'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.copy()
    df = df.dropna(subset=["tags"]).reset_index(drop=True)
    # replace int with empty list
    df['tags'] = df['tags'].apply(lambda x: x if x else [])

    # Precompute the similarity between each tag word and each preference word
    similarity_dict = {}
    for tag_word in set([word for tags in df['tags'] for word in tags]):
        for pref_word in set(preferences):
            similarity_dict[(tag_word, pref_word)] = nlp(tag_word).similarity(nlp(pref_word))

    # Compute the average similarity for each row in the dataframe
    similarities = []
    for tags in df['tags']:
        sum_similarity = 0
        for tag_word in tags:
            for pref_word in preferences:
                sum_similarity += similarity_dict[(tag_word, pref_word)]
        avg_similarity = sum_similarity / (len(tags) * len(preferences)) if len(tags) > 0 else 0
        similarities.append(avg_similarity)

    # Add the similarity scores to a new column in the dataframe
    df['similarity_tags'] = similarities
    if n == 0:
        closest_matches = df.sort_values('similarity_tags', ascending=False)[
            ['filename', 'similarity_tags']]
    else:
        closest_matches = df.sort_values('similarity_tags', ascending=False).head(n)[
            ['filename', 'similarity_tags']]

    return closest_matches

In [ ]:
recommend_tags(df_metadata, df_preferences)  # OK

# Make Similarity
# recommend_make() Function

The `recommend_make()` function is used to recommend images based on the make of a camera used to take the image. This function takes three parameters as input:
1. `df_metadata`: a DataFrame containing metadata of images
2. `df_preferences`: a DataFrame containing the make of the camera that the user wants to find similar images to
3. `n`: an optional parameter that indicates the number of images to be recommended.

The function starts by loading the spaCy NLP model "en_core_web_md". Then it defines the camera make from the preferences DataFrame, and loads the metadata DataFrame and removes the rows containing NaN values in the "Make" column.

The function converts the camera make preference and the "Make" column of the metadata DataFrame into spaCy document objects. Then it calculates the cosine similarity between the make preference and all the makes in the metadata DataFrame.

The calculated similarity scores are added to a new column "similarity_make" in the metadata DataFrame. The metadata DataFrame is then sorted based on the similarity scores and either the top `n` rows or all the rows (if n=0) are returned as the recommended images.


In [ ]:
def recommend_make(df_metadata, df_preferences, n=0):
    # Load the spaCy model
    nlp = spacy.load("en_core_web_md")

    # Define the preferences list and the dataframe
    make = df_preferences['Make'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.copy()
    df = df.dropna(subset=["Make"]).reset_index(drop=True)

    # Convert make and Make to document objects
    make_doc = nlp(make)
    df['Make'] = df['Make'].apply(nlp)

    # Compute the cosine similarity between the make preferences and all the makes in the dataset
    similarities = [make_doc.similarity(doc) for doc in df['Make']]

    # Add the similarity scores to a new column in the dataframe
    df['similarity_make'] = similarities
    if n == 0:
        closest_matches = df.sort_values('similarity_make', ascending=False)[
            ['filename', 'similarity_make']]
    else:
        closest_matches = df.sort_values('similarity_make', ascending=False).head(n)[
            ['filename', 'similarity_make']]

    return closest_matches


In [ ]:
recommend_make(df_metadata, df_preferences)  # OK

# Orientation Similarity

# Function to Recommend Orientation

The `recommend_orientation` function takes in three arguments:
1. `df_metadata`: A pandas dataframe containing metadata of images.
2. `df_preferences`: A pandas dataframe containing the user's preferences for orientation.
3. `n`: Number of recommendations to return. (default is 0, meaning all)

## Step 1: Define the preferences
The user's preferred orientation is extracted from the `df_preferences` dataframe.

## Step 2: Load the dataset
The `df_metadata` dataframe is loaded into a new dataframe, `df`, and any rows with missing values in the "Orientation" column are removed.

## Step 3: Clean the Orientation column
The "Orientation" column is converted to integer type, and any values of '' or '0' are replaced with 0, and any other values are replaced with 1.

## Step 4: Calculate similarity
The absolute difference between the "Orientation" column and the user's preferred orientation is calculated and stored in a new column, "similarity_orientation".

## Step 5: Sort and return recommendations
The dataframe is sorted by "similarity_orientation" in ascending order and the first `n` rows are returned. If `n` is 0 (default), all rows are returned.


In [ ]:
def recommend_orientation(df_metadata, df_preferences, n=0):
    # Define the preferences list and the dataframe
    orientation = df_preferences['Orientation'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.dropna(subset=["Orientation"]).reset_index(drop=True)
    # if Orientation contain '' or '0' or '1' then replace with 0 or 1
    df['Orientation'] = df['Orientation'].apply(lambda x: 0 if x == '' or x == '0' else 1)

    # Convert the Orientation column to integer type
    df['Orientation'] = df['Orientation'].astype(int)

    # Orientation is 0 or 1, so we can just subtract the preference from the orientation
    df['similarity_orientation'] = df['Orientation'].apply(lambda x: abs(x - orientation))

    # sort by similarity
    if n > 0:
        closest_matches = df.sort_values('similarity_orientation', ascending=False).head(n)[
            ['filename', 'similarity_orientation']]
    else:
        closest_matches = df.sort_values('similarity_orientation', ascending=False)[
            ['filename', 'similarity_orientation']]

    return closest_matches


In [ ]:
recommend_orientation(df_metadata, df_preferences)  # OK

# Size Similarity

The `recommend_size` function is used to recommend images based on their size. The function takes three arguments: `df_metadata`, `df_preferences`, and `n`, where `df_metadata` is the metadata of all images, `df_preferences` is the preferred size of the image, and `n` is the number of recommended images to return.

The function starts by defining two variables `width` and `height` from the `df_preferences` dataframe and then loads the metadata of all images into a dataframe `df`. It removes any rows in the `df` dataframe that contain NaN values in the `ImageWidth` or `ImageHeight` columns.

Next, the function converts the `ImageWidth` and `ImageHeight` columns in the `df` dataframe to integer type. Then, it computes the product of the `width` and `height` variables and stores it in a variable `product`.

The function then uses the `apply` method to compute a similarity score for each row in the `df` dataframe. The similarity score is calculated by subtracting the product of the image width and height from the preferred `product` and dividing the result by the `product`.

Finally, the function sorts the `df` dataframe based on the similarity score in descending order and returns the top `n` rows if `n` is not equal to zero, or returns all rows if `n` is equal to zero. The returned dataframe contains two columns: `filename` and `similarity_size`.



In [ ]:
def recommend_size(df_metadata, df_preferences, n=0):
    # Define the preferences list and the dataframe
    width = int(df_preferences['ImageWidth'][0])
    height = int(df_preferences['ImageHeight'][0])
    # Load dataset with words and drop duplicate rows
    df = df_metadata.dropna(subset=["ImageWidth", "ImageHeight"]).reset_index(drop=True)

    # Convert the ImageWidth and ImageHeight column to integer type
    df[['ImageWidth', 'ImageHeight']] = df[['ImageWidth', 'ImageHeight']].astype(int)

    # Compute the product of width and height outside the loop
    product = width * height

    # Use apply method to compute similarity score for each row
    df['similarity_size'] = df.apply(lambda x: 1 - abs(product - (x['ImageWidth'] * x['ImageHeight'])) / product, axis=1)

    if n == 0:
        closest_matches = df.sort_values('similarity_size', ascending=False)[
            ['filename', 'similarity_size']]
    else:
        closest_matches = df.sort_values('similarity_size', ascending=False).head(n)[
            ['filename', 'similarity_size']]

    return closest_matches


In [ ]:
recommend_size(df_metadata, df_preferences)  # OK

# Recommendation Method

The `recommend` function is a recommendation method that computes the similarity score between a set of images and a set of preferences. The function combines the results of several other recommendation methods, which are each designed to compare the images with one aspect of the preferences. The final similarity score is the weighted average of the scores from the individual methods.

## Inputs

- `df_metadata`: A Pandas DataFrame that contains metadata for each image.
- `df_preferences`: A Pandas DataFrame that contains the preferences of the user.
- `n`: The number of images to return in the result. If `n` is set to 0, all images will be returned.

## Output

- A Pandas DataFrame that contains the `filename` of each image, along with the `similarity_score` between the image and the user preferences. The images are sorted in descending order of similarity score.

## Implementation Details

1. Assign weights to the different properties based on the user preferences.
2. Create a dictionary with the preferences and the corresponding recommendation methods.
3. Remove preferences with no values.
4. Calculate the sum of the weights.
5. Calculate the similarity score for each property using the corresponding recommendation method and weighting the scores.
6. Replace NaN values in the `similarity_score` column with 0.
7. Sort by similarity score and return the top `n` images.

In [ ]:
def recommend(df_metadata, df_preferences, n=0):
    # Assign weights to properties based on user preferences
    weights = {
        'Make': float(5.0),
        'ImageWidth': float(1.0),
        'ImageHeight': float(1.0),
        'Orientation': float(2.0),
        'dominant_color': float(3.0),
        'tags': float(5.0)
    }

    # Create a dictionary with the preferences and the corresponding recommendation methods
    preference_methods = {
        'Make': recommend_make,
        'ImageWidth': recommend_size,
        'ImageHeight': recommend_size,
        'Orientation': recommend_orientation,
        'dominant_color': recommend_colors,
        'tags': recommend_tags
    }

    # Remove preferences with no values
    preferences = {k: v for k, v in df_preferences.squeeze().to_dict().items() if v != ''}

    # Calculate the sum of the weights
    weights_sum = 0
    for key in weights:
        weights_sum += weights[key]
    for key in weights:
        weights[key] = weights[key] / weights_sum

    # Calculate similarity score for each property
    df_metadata['similarity_score'] = 0.0
    for preference, value in preferences.items():
        method = preference_methods[preference]
        if preference == 'ImageWidth' or preference == 'ImageHeight':
            similarity = method(df_metadata, df_preferences, n)['similarity_size'].astype(float)
        else:
            similarity = method(df_metadata, df_preferences, n)[f'similarity_{preference.lower()}'].astype(float)
        df_metadata['similarity_score'] += similarity * (weights[preference] / weights_sum)

    # Replace NaN values in the 'similarity_score' column with 0
    df_metadata['similarity_score'].fillna(0, inplace=True)

    # Sort by similarity score
    if n == 0:
        closest_matches = df_metadata.sort_values('similarity_score', ascending=False)[
            ['filename', 'similarity_score']]
    else:
        closest_matches = df_metadata.sort_values('similarity_score', ascending=False).head(n)[
            ['filename', 'similarity_score']]

    return closest_matches


In [ ]:
recommend(df_metadata, df_preferences)  # OK

# Recommend by grouping all properties
# Data Retrieval and User Preference Definition

The code retrieves image metadata as image properties using the `get_metadata_as_imageproperties()` method. Then, it defines a dictionary `preferences` to specify the user's preferred values for the properties of an image such as the 'Make' of the camera used, the 'ImageWidth', 'ImageHeight', 'Orientation', the 'dominant_color', and the 'tags' associated with the image.

The values specified in the `preferences` dictionary are as follows:
- 'Make': 'Canon'
- 'ImageWidth': ''
- 'ImageHeight': ''
- 'Orientation': 1
- 'dominant_color': '#00000'
- 'tags': ['cat']

Note that for 'ImageWidth' and 'ImageHeight', no values are specified, indicated by an empty string.


In [ ]:
data = get_metadata_as_imageproperties()

preferences = {
    'Make': 'Canon',
    'ImageWidth': '',
    'ImageHeight': '',
    'Orientation': 1,
    'dominant_color': '#00000',
    'tags': ['cat']
}


# Function to Preprocess Data

The `preprocess_data` function takes a data parameter which is a list of image properties. The function processes each image property in the data list and returns an array of processed data for each image. The function performs the following tasks for each image in the data list:

1. It joins the tags of the image into a single string.
2. It calculates the average RGB color of the image based on the list of hexadecimal color codes.
3. It tries to convert the make of the image into an integer, and if it is not possible, it sets the make length to 0.
4. It tries to convert the width and height of the image into integers, and if it is not possible, it sets them to 0.
5. It tries to convert the orientation of the image into an integer, and if it is not possible, it sets the orientation to 0.
6. It appends the average RGB color, tag length, make length, orientation, width, and height of the image to an array.
7. It returns the array for each image in the data list.

Note that if any error occurs while processing the data for an image, that image's data is skipped and the function moves on to the next image.


In [ ]:
def preprocess_data(data):
    for image in data:
        image_tags = " ".join(image.tags)
        avg_rgb_color = np.mean([hex_to_rgb(color) for color in image.hex_color], axis=0)
        try:
            make_len = len(image.make)
        except TypeError:
            make_len = 0

        try:
            width = int(image.width)
        except TypeError:
            width = 0

        try:
            height = int(image.height)
        except TypeError:
            height = 0

        try:
            orientation = int(image.orientation)
        except:
            orientation = 0

        avg_rgb_color_list = avg_rgb_color.tolist() if hasattr(avg_rgb_color, 'tolist') else [avg_rgb_color]

        try:
            yield np.array([
                *avg_rgb_color_list,  # Ensure avg_rgb_color is an iterable before unpacking
                len(image_tags),
                make_len,
                orientation,
                width,
                height
            ])
        except:
            pass

This code defines a function user_preferences_vector which takes in a dictionary of preferences as input and returns a numpy array containing the processed data. The input preferences include the following properties:

- dominant_color: The preferred dominant color in hexadecimal format.
- tags: A list of preferred tags.
- Make: The preferred make.
- Orientation: The preferred orientation.
- ImageWidth: The preferred image width.
- ImageHeight: The preferred image height.

The function first converts the dominant_color from hexadecimal format to RGB format using the hex_to_rgb function. Then, it converts the list of preferred tags into a string by joining the elements of the list. The length of the Make and the tags string are also calculated.

The Orientation, ImageWidth, and ImageHeight values are then processed to make sure they are in the correct format (integers).

Finally, the processed data is returned as a numpy array.

In [ ]:
def user_preferences_vector(preferences):
    dominant_color = hex_to_rgb(preferences['dominant_color'])
    tags = " ".join(preferences['tags'])
    make = preferences['Make']

    try:
        make_len = len(make)
    except TypeError:
        make_len = 0

    orientation = preferences['Orientation']

    try:
        width = int(preferences['ImageWidth'])
    except (TypeError, ValueError):
        width = 0

    try:
        height = int(preferences['ImageHeight'])
    except (TypeError, ValueError):
        height = 0

    return np.array([
        *dominant_color,
        len(tags),
        make_len,
        orientation,
        width,
        height
    ])

# Function to recommend Images based on User Preferences

The `recommend_images` function takes in three parameters: `preferences`, `data`, and `top_n`. The `preferences` parameter is a dictionary containing the user's preferences for the image properties, `data` is a list of image properties, and `top_n` is an optional parameter specifying the number of images to be returned.

The function starts by preprocessing the `data` using the `preprocess_data` function, converting the image properties into a numerical representation. It then computes a user preferences vector using the `user_preferences_vector` function, which takes the `preferences` dictionary as input and returns a numerical representation of the user's preferences.

The cosine similarity between the user vector and each of the preprocessed image properties is then calculated using the `cosine_similarity` function from the scikit-learn library. The `cosine_similarity` function returns a similarity matrix, with the first row containing the similarity scores for the user vector and the other rows containing the similarity scores for each preprocessed image property.

The indices of the top `top_n` most similar images are then obtained using the `np.argsort` function, which sorts the similarity scores in ascending order and returns the indices of the elements in the sorted array. The final step is to return the actual image properties for the top `top_n` most similar images by indexing into the original `data` list.

In [ ]:
def recommend_images(preferences, data, top_n=10):
    preprocessed_data = np.array(list(preprocess_data(data)))
    user_vector = user_preferences_vector(preferences)
    similarity_matrix = cosine_similarity(np.vstack([user_vector, preprocessed_data]))
    most_similar_indices = np.argsort(-similarity_matrix[0])[1:top_n+1]
    return [data[i - 1] for i in most_similar_indices]

In [ ]:
# Test the recommender system
recommended_images = recommend_images(preferences, data)
for image in recommended_images:
    print(image.name)

# Recommend images with RL

## Explanation of the `clean` Function

The `clean` function is responsible for cleaning and preparing a given pandas DataFrame called `data`. The function follows these steps:

1. **Extract the first hex color**: The `hex_color` column contains a list of hex colors. The function extracts the first color in the list by applying a lambda function on the 'hex_color' column. If the list is empty, it assigns `None` to the value.
2. **Convert columns to the correct data type**: The function converts the 'width', 'height', and 'orientation' columns to numeric types using the `pd.to_numeric` function. If there are any errors during conversion, the function coerces the values to NaN.
3. **Fill missing values**: It fills the missing values (NaN) in the 'width', 'height', and 'orientation' columns with 0 using the `fillna` function.
4. **Remove rows with missing values**: Finally, the function removes rows containing any missing values using the `dropna` function.

After these steps, the function returns the cleaned DataFrame.

In [ ]:
def clean(data):
    """
    :param data: DataFrame
    :return:
    """
    # 1. first, get only the first hex_color
    # 2. Convert each column to the right type of data
    # 2. Remove rows with missing values

    data['hex_color'] = data['hex_color'].apply(lambda x: eval(x)[0] if len(x) > 0 else None)
    data['width'] = pd.to_numeric(data['width'], errors='coerce')
    data['height'] = pd.to_numeric(data['height'], errors='coerce')
    data['orientation'] = pd.to_numeric(data['orientation'], errors='coerce')
    data['width'].fillna(0, inplace=True)
    data['height'].fillna(0, inplace=True)
    data['orientation'].fillna(0, inplace=True)

    data = data.dropna()

    return data

## `preprocess` Function

The `preprocess` function is responsible for preprocessing a given pandas DataFrame called `data`. The function performs the following steps:

1. **Normalize and scale numerical properties**: The 'width' and 'height' columns are scaled to the range [0, 1] by dividing each value by the maximum value in the respective column. The 'hex_color' column is converted to an RGB representation and normalized by dividing each component by 255, resulting in values in the range [0, 1].
   - The '#' symbol is removed from the 'hex_color' column.
   - The 'hex_color' values are converted to RGB tuples and stored in the 'rgb_color' column.
   - The RGB components are separated into the 'r', 'g', and 'b' columns, and the 'hex_color' and 'rgb_color' columns are dropped.

2. **One-hot encode categorical properties**: One-hot encoding is applied to the 'tags', 'make', and 'orientation' columns.
   - The 'tags' column is first converted into an array using the `ast.literal_eval` function.
   - One-hot encoding is applied to the 'tags' array by stacking the array elements into a single column, applying `pd.get_dummies`, and summing the resulting dummy columns at the original DataFrame index level.
   - One-hot encoding is applied to the 'make' column using the `pd.get_dummies` function with the 'make' prefix.
   - The 'orientation' column is one-hot encoded into 'landscape' and 'portrait' columns, where 'landscape' is 1 if the orientation is 0 and 'portrait' is 1 if the orientation is 1.
   - The original 'tags', 'make', and 'orientation' columns are dropped.

After these preprocessing steps, the function returns the preprocessed DataFrame.

In [ ]:
def preprocess(data):
    # 1. Normalize and scale numerical properties
    # Width and Height: scale to [0, 1] by dividing each value by the maximum value
    # Hex colors: Convert hex colors to RGB values in the range 0-255 and normalize it by dividing by 255 (range [0, 1])

    data['width'] = data['width'] / data['width'].max()
    data['height'] = data['height'] / data['height'].max()

    # Convert hex color to RGB and normalize
    # Remove # from hex color
    data["hex_color"] = data["hex_color"].apply(lambda x: x[1:])
    data["rgb_color"] = data["hex_color"].apply(lambda x: tuple(int(x[i:i + 2], 16) for i in (0, 2, 4)))
    data[["r", "g", "b"]] = pd.DataFrame(data["rgb_color"].tolist(), index=data.index) / 255
    data.drop(["hex_color", "rgb_color"], axis=1, inplace=True)

    # One-hot encode categorical properties
    # 1. tags, make, orientation
    # eval tags to have an array
    convert_to_array = lambda tag_string: ast.literal_eval(tag_string)

    data["tags"] = data["tags"].apply(convert_to_array)
    data = pd.concat([data, pd.get_dummies(data["tags"].apply(pd.Series).stack()).sum(level=0)], axis=1)
    data = pd.concat([data, pd.get_dummies(data["make"], prefix="make")], axis=1)
    data["landscape"] = data["orientation"].apply(lambda x: 1 if x == 0 else 0)
    data["portrait"] = data["orientation"].apply(lambda x: 1 if x == 1 else 0)
    data.drop(["tags", "make", "orientation"], axis=1, inplace=True)

    return data

## `train_and_test` Function

The `train_and_test` function is responsible for training and evaluating a deep Q-network (DQN) agent for an image recommendation task. The function follows these steps:

1. **Load and preprocess data**: Load the raw image properties from a CSV file and preprocess the data using the `clean` and `preprocess` functions. Fill any remaining missing values with 0.
2. **Create feature vectors**: Drop the 'name' column and convert the DataFrame into a numpy array to create feature vectors.
3. **Combine feature vectors with image names**: Create a new DataFrame containing the image names and their corresponding feature vectors.
4. **Split the data**: Split the dataset into training (70%), validation (15%), and test (15%) sets using the `train_test_split` function.
5. **Create environments**: Instantiate a `DummyVecEnv` for the training set and create validation and test environments using the `ImageRecommenderEnvironment` class.
6. **Define and train the DQN agent**: Create a DQN agent with custom settings, such as the learning rate, buffer size, batch size, gamma, and tau values. Train the agent for a predefined number of timesteps.
7. **Save and load the agent**: Save the trained agent to a file and load it for evaluation.
8. **Evaluate the agent**: Evaluate the agent on the validation and test sets by running a fixed number of episodes and calculating the average reward, duration, and number of steps.
9. **Print evaluation results**: Display the evaluation results, including reward, duration, and steps for both the validation and test sets.

The function calculates the precision, recall, and F1-score for the agent's performance, but these metrics are not yet implemented in the code.

In [ ]:
def train_and_test():
    df = pd.read_csv('./raw_image_properties.csv', sep="|")

    # Convert the list of ImageProperties objects to a pandas DataFrame
    #df = pd.DataFrame(raw_image_properties)

    clean_image_properties = clean(df)
    processed_image_properties = preprocess(clean_image_properties)

    # Check for NaN values in the DataFrame
    if processed_image_properties.isnull().values.any():
        processed_image_properties.fillna(0, inplace=True)

    # Create feature vectors (numpy array)
    feature_vectors = processed_image_properties.drop(["name"], axis=1).values

    # Combine the feature vectors with the image names in a new DataFrame
    data_with_features = processed_image_properties[['name']].copy()
    data_with_features['features'] = list(feature_vectors)

    # Split the dataset into training (70%) and the remaining 30%
    train_data, temp_data = train_test_split(data_with_features, test_size=0.3, random_state=42)

    # Split the remaining data into validation (15%) and test (15%) sets
    validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    env = DummyVecEnv([lambda: ImageRecommenderEnvironment(data_with_features)])

    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    policy_kwargs = dict(
        features_extractor_class=CustomDQNNetwork,
        features_extractor_kwargs={'input_dim': input_dim, 'output_dim': output_dim}
    )

    agent = DQN(MlpPolicy,
                env,
                policy_kwargs=policy_kwargs,
                learning_rate=1e-3,
                buffer_size=100000,
                batch_size=64,
                gamma=0.99,
                tau=0.01,
                verbose=0)

    total_timesteps = 500000  # Adjust this value based on the problem and computational resources
    print("Starting training at time", time.strftime("%H:%M:%S", time.localtime()))
    start_time = time.time()
    agent.learn(total_timesteps=total_timesteps)
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds")

    # Save the trained agent
    agent.save("dqn_agent.pkl")

    # Load the trained agent
    #agent = DQN.load("dqn_agent.pkl")

    # Create validation and test environments
    validation_env = DummyVecEnv([lambda: ImageRecommenderEnvironment(validation_data)])
    test_env = DummyVecEnv([lambda: ImageRecommenderEnvironment(test_data)])

    # Evaluate the agent on the validation and test sets
    num_validation_episodes = 100000
    num_test_episodes = 100000

    print("Starting evaluation at time", time.strftime("%H:%M:%S", time.localtime()))
    start_time = time.time()
    validation_reward, validation_duration, validation_steps = evaluate_agent(agent, validation_env, num_validation_episodes)
    test_reward, test_duration, test_steps = evaluate_agent(agent, test_env, num_test_episodes)
    end_time = time.time()
    print(f"Evaluation completed in {end_time - start_time:.2f} seconds")

    print(f"Validation reward: {validation_reward}, duration: {validation_duration:.2f}s, steps: {validation_steps}")
    print(f"Test reward: {test_reward}, duration: {test_duration:.2f}s, steps: {test_steps}")

    # Precision = TP / (TP + FP)
    # Recall = TP / (TP + FN)
    # F1-score = 2 * (Precision * Recall) / (Precision + Recall)

## Explanation of the `preprocess_preferences` Function

The `preprocess_preferences` function is responsible for preprocessing a given pandas DataFrame called `preferences`, which represents user preferences for image recommendations. The function performs the following steps:

1. **Normalize and scale numerical properties**: Scale 'width' and 'height' columns to the range [0, 1] by dividing each value by the maximum width and height passed as arguments. Convert 'hex_color' to normalized RGB values in the range [0, 1].
   - Remove the '#' symbol from the 'hex_color' column.
   - Convert 'hex_color' values to RGB tuples and store them in the 'rgb_color' column.
   - Separate RGB components into the 'r', 'g', and 'b' columns, then drop the 'hex_color' and 'rgb_color' columns.

2. **One-hot encode categorical properties**: One-hot encode the 'tags', 'make', and 'orientation' columns.
   - Convert the 'tags' column into an array using the `ast.literal_eval` function.
   - One-hot encode the 'tags' array by stacking its elements into a single column, applying `pd.get_dummies`, and summing the resulting dummy columns at the original DataFrame index level.
   - One-hot encode the 'make' column using the `pd.get_dummies` function with the 'make' prefix.
   - One-hot encode the 'orientation' column into 'landscape' and 'portrait' columns, where 'landscape' is 1 if the orientation is 0 and 'portrait' is 1 if the orientation is 1.
   - Drop the original 'tags', 'make', and 'orientation' columns.

3. **Ensure consistent column order**: Compare the processed `preferences` DataFrame with a list of all possible column indexes (`all_indexes`) and adjust the DataFrame accordingly.
   - Add missing columns from `all_indexes` to the DataFrame and set their values to 0.
   - Remove any extra columns from the DataFrame that are not in `all_indexes`.
   - If the number of columns in the processed DataFrame is not equal to the length of `all_indexes`, drop any columns not in `all_indexes` and add missing columns with values set to 0.

After these preprocessing steps, the function returns the preprocessed user preferences DataFrame.

In [ ]:
def preprocess_preferences(preferences, max_width, max_height, all_indexes):
        # 1. Normalize and scale numerical properties
        # Width and Height: scale to [0, 1] by dividing each value by the maximum value
        # Hex colors: Convert hex colors to RGB values in the range 0-255 and normalize it by dividing by 255 (range [0, 1])

        preferences['width'] = preferences['width'] / max_width
        preferences['height'] = preferences['height'] / max_height

        # Convert hex color to RGB and normalize
        # Remove # from hex color
        try:
            preferences["hex_color"] = preferences["hex_color"].apply(lambda x: x[1:])
            preferences["rgb_color"] = preferences["hex_color"].apply(lambda x: tuple(int(x[i:i + 2], 16) for i in (0, 2, 4)))
            preferences[["r", "g", "b"]] = pd.DataFrame(preferences["rgb_color"].tolist(), index=preferences.index) / 255
            preferences.drop(["hex_color", "rgb_color"], axis=1, inplace=True)
        except:
            pass

        # One-hot encode categorical properties
        # 1. tags, make, orientation
        # eval tags to have an array
        convert_to_array = lambda tag_string: ast.literal_eval(tag_string)

        try:
            preferences["tags"] = preferences["tags"].apply(convert_to_array)
            data = pd.concat([preferences, pd.get_dummies(preferences["tags"].apply(pd.Series).stack()).sum(level=0)], axis=1)
        except:
            pass
        try:
            data = pd.concat([data, pd.get_dummies(data["make"], prefix="make")], axis=1)
        except:
            data["make"] = 0
        data["landscape"] = data["orientation"].apply(lambda x: 1 if x == 0 else 0)
        data["portrait"] = data["orientation"].apply(lambda x: 1 if x == 1 else 0)
        data.drop(["tags", "make", "orientation"], axis=1, inplace=True)

        present_indexes = data.columns.values.tolist()

        # Add all indexes to the data as columns and set it to 0 if it is not present
        for index in all_indexes:
            if index not in present_indexes:
                data[index] = 0

        try:
            data.drop("hex_color")
        except:
            pass

        # if prepo_preferences do not have the same lenght as processed_image_properties remove columns that is not in processed_image_properties and add the one that are not in  prepro_preferences bt processed_image_properties

        if len(data.columns) != len(all_indexes):
            for col in data.columns:
                if col not in all_indexes:
                    data.drop(col, axis=1, inplace=True)
            for col in data.columns:
                if col not in data.columns:
                    data[col] = 0

        return data

## `clean_prefs` Function

The `clean_prefs` function is responsible for cleaning and handling missing or erroneous values in a given pandas DataFrame called `preferences`, which represents user preferences for image recommendations. The function performs the following steps:

1. **Clean 'hex_color' column**: Extract the first hex color from the 'hex_color' column. If the column is empty or an error occurs, set the value to 0.

2. **Convert columns to numeric**: Convert the 'width', 'height', and 'orientation' columns to numeric values, using the `pd.to_numeric` function with the 'coerce' error handling option. If an error occurs, set the respective value to None.
   - For the 'width' column, set any error-causing values to None.
   - For the 'height' column, set any error-causing values to None.
   - For the 'orientation' column, set any error-causing values to None.

3. **Fill missing values**: Fill any missing values (None) in the 'width', 'height', and 'orientation' columns with 0, using the `fillna` function with the 'inplace' option set to True.

After these cleaning steps, the function returns the cleaned user preferences DataFrame.


In [ ]:
def clean_prefs(preferences):

    try:
        preferences['hex_color'] = preferences['hex_color'].apply(lambda x: eval(x)[0] if len(x) > 0 else None)
    except:
        preferences['hex_color'] = 0

    try:
        preferences['width'] = pd.to_numeric(preferences['width'], errors='coerce')
    except:
        preferences['width'] = None

    try:
        preferences['height'] = pd.to_numeric(preferences['height'], errors='coerce')
    except:
        preferences['height'] = None

    try:
        preferences['orientation'] = pd.to_numeric(preferences['orientation'], errors='coerce')
    except:
        preferences['orientation'] = None

    preferences['width'].fillna(0, inplace=True)
    preferences['height'].fillna(0, inplace=True)
    preferences['orientation'].fillna(0, inplace=True)

    # check if there is make or others to set it to None
    return preferences

In [ ]:
def get_image_name(image_filenames, index):
    return image_filenames[index]

## `recommend` Function

The `recommend` function serves as an image recommendation engine. It processes user preferences and returns a recommended image based on those preferences. Here is a step-by-step explanation of the function:

1. **Load image properties**: Read the raw image properties from a CSV file and store them in a pandas DataFrame called `df`.

2. **Clean and preprocess image properties**: Clean the image properties using the `clean` function, and then preprocess the cleaned properties using the `preprocess` function. Store the maximum width and height values for later use.

3. **Create user preferences**: Create a dictionary with user preferences, including 'make', 'tags', 'hex_color', 'width', and 'height'.

4. **Convert user preferences to DataFrame**: Convert the user preferences dictionary to a pandas DataFrame called `df`.

5. **Clean and preprocess user preferences**: Clean the user preferences using the `clean_prefs` function, and preprocess the cleaned preferences using the `preprocess_preferences` function. The preprocessed preferences are adjusted to match the format of the processed image properties.

6. **Load the trained model**: Load the previously trained DQN model from the 'dqn_agent.pkl' file.

7. **Predict the recommended image**: Drop the first column from the preprocessed preferences DataFrame. Then, use the DQN model to predict the recommended image index based on the user preferences, setting the `deterministic` parameter to `True`.

8. **Return the recommended image**: Use the `get_image_name` function to retrieve the name of the recommended image from the list of image filenames, and return it as the output of the `recommend` function.


In [ ]:
def recommend():

    df = pd.read_csv('./raw_image_properties.csv', sep="|")

    # Convert the list of ImageProperties objects to a pandas DataFrame
    #df = pd.DataFrame(raw_image_properties)

    clean_image_properties = clean(df)
    max_width = clean_image_properties["width"].max()
    max_height = clean_image_properties["height"].max()
    processed_image_properties = preprocess(clean_image_properties)

    images_filenames = processed_image_properties["name"].values.tolist()

    tags = "['dog']"

    preferences = {
        'make': "Nikon",
        'tags': tags,
        'hex_color': "#88313",
        'width': 2000,
        'height': 3000
    }

    df = pd.DataFrame([preferences])

    clean_preferences = clean_prefs(df)

    prepro_preferences = preprocess_preferences(clean_preferences, max_width, max_height, processed_image_properties.columns)

    # do the prediction by loading the model
    model = DQN.load("dqn_agent.pkl")
    # drop the first column
    prepro_preferences.drop(prepro_preferences.columns[0], axis=1, inplace=True)
    action, _ = model.predict(prepro_preferences, deterministic=True)

    return get_image_name(images_filenames, int(action))

## `ImageRecommenderEnvironment` Class

The `ImageRecommenderEnvironment` class is a custom Gym environment for the image recommendation task. It inherits from `gym.Env`, and it is designed to work with the DQN agent. The key components of the class are as follows:

1. **Initialization**: The `__init__` method initializes the environment with the given data, setting the action space to the indices of the images and the state space to the feature vectors. The maximum number of steps per episode is also specified.

2. **Reset**: The `reset` method resets the environment to an initial state by randomly selecting an image from the data.

3. **Step**: The `step` method executes the given action, updates the environment's state, and calculates the reward. The next state and reward are determined using the `_get_next_state_and_reward` method. The method also checks if the episode is done using the `_is_done` method.

4. **Render**: The `render` method is optional and can be implemented to visualize the current state of the environment.

5. **_get_initial_state**: This helper method randomly selects an initial state from the data.

6. **_get_next_state_and_reward**: This helper method takes the action as input, limits the action value to the maximum index of the DataFrame, retrieves the recommended image's features, calculates the reward using the `reward_function`, and returns the next state (which is the same as the current state in this implementation) and the reward.

7. **_is_done**: This helper method checks if the episode is done based on the specified condition, which is reaching the maximum number of steps in this example.


In [ ]:
class ImageRecommenderEnvironment(gym.Env):
    def __init__(self, data_with_features, max_steps=100):
        super(ImageRecommenderEnvironment, self).__init__()

        self.data = data_with_features
        self.current_state = None
        self.max_steps = max_steps
        self.step_count = 0

        # Define action space as the indices of the images
        self.action_space = spaces.Discrete(len(self.data))

        # Define state space as the feature vectors
        feature_vector_length = len(self.data['features'].iloc[0])
        self.observation_space = spaces.Box(low=0, high=1, shape=(feature_vector_length,), dtype=np.float32)

    def reset(self):
        # Reset the environment to an initial state
        self.current_state = self._get_initial_state()
        return self.current_state

    def step(self, action):
        # Execute the given action and observe the next state and reward
        next_state, reward = self._get_next_state_and_reward(action)

        # Check if the episode is done
        self.step_count += 1
        done = self._is_done(next_state)

        # Update the current state
        self.current_state = next_state

        return next_state, reward, done, {}

    def render(self, mode='human'):
        # Render the current state of the environment (optional)
        pass

    def _get_initial_state(self):
        # Randomly select an initial state from the data
        initial_state = self.data.sample().iloc[0]['features']
        return initial_state

    def _get_next_state_and_reward(self, action):
        # Limit the action value to the maximum index of the DataFrame
        action = min(action, len(self.data) - 1)

        # Get the recommended image's features based on the action
        recommended_image_features = self.data.iloc[action]['features']

        # Calculate the reward using the reward_function (you need to define user_preferences)
        user_preferences = self.current_state.reshape(1, -1)
        recommended_image_features = recommended_image_features.reshape(1, -1)
        reward = reward_function(user_preferences, recommended_image_features)

        # Get the next state (in this case, we will use the same state as the current state)
        next_state = self.current_state

        return next_state, reward


    def _is_done(self, next_state):
        # In this example, we will define a specific condition for the episode to be done.
        # If the step_count reaches the max_steps, the episode is done.
        return self.step_count >= self.max_steps

## `CustomDQNNetwork` Class

The `CustomDQNNetwork` class is a custom neural network module that inherits from PyTorch's `nn.Module`. It is designed to work with the DQN agent, and its main components are as follows:

1. **Initialization**: The `__init__` method initializes the neural network with the given input and output dimensions. It defines a simple feedforward neural network architecture using the `nn.Sequential` module. The network comprises three fully connected (`nn.Linear`) layers with ReLU activation functions and dropout layers with a dropout rate of 0.5 for regularization.

2. **Forward Pass**: The `forward` method is responsible for the forward pass of the input through the network. It takes the input tensor `x` and passes it through the defined network architecture, returning the output tensor.

The `CustomDQNNetwork` class serves as the feature extractor for the DQN agent, and its architecture can be modified as needed to improve the performance of the agent on the image recommendation task.


In [ ]:
class CustomDQNNetwork(nn.Module):
    def __init__(self, *args, input_dim=None, output_dim=None, **kwargs):
        super(CustomDQNNetwork, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, output_dim)
        )

        self.features_dim = output_dim

    def forward(self, x):
        return self.network(x)

##  `reward_function`

The `reward_function` is used to calculate the reward for a given recommendation based on the user preferences and the features of the recommended image. It is an essential component of the reinforcement learning environment. The function takes two input arguments: `user_preferences` and `recommended_image_features`, both of which are 1-dimensional feature vectors. The function performs the following steps:

1. **Flatten Input Vectors**: The function checks whether the input vectors have the correct dimensions (1-dimensional). If not, it flattens the input vectors to ensure they are 1-dimensional.

2. **Calculate Euclidean Distance**: The function computes the Euclidean distance between the user preferences and the recommended image features. This distance represents how dissimilar the recommendation is to the user's preferences.

3. **Calculate Reward**: The reward is calculated as the inverse of the sum of 1 and the calculated distance. This formula normalizes the distance to a range between 0 and 1, where a value closer to 1 represents a better match between the recommendation and user preferences, and a value closer to 0 indicates a poor match.

The `reward_function` plays a crucial role in guiding the agent's learning process, as it informs the agent about the quality of its recommendations based on the user's preferences.


In [ ]:
def reward_function(user_preferences, recommended_image_features):
    # Ensure that input vectors are 1-dimensional
    if not (np.ndim(user_preferences) == 1 and np.ndim(recommended_image_features) == 1):
        user_preferences = np.asarray(user_preferences).flatten()
        recommended_image_features = np.asarray(recommended_image_features).flatten()

    distance = euclidean(user_preferences, recommended_image_features)
    reward = 1 / (1 + distance)  # Normalize the distance to [0, 1] by using the inverse
    return reward

## `evaluate_agent` Function

The `evaluate_agent` function is used to evaluate the performance of a trained reinforcement learning agent in a given environment. The function takes three input arguments: `agent`, `env`, and `num_episodes`. The function performs the following steps:

1. **Initialize Metrics**: It initializes empty lists for the total rewards, episode durations, and episode steps to keep track of the agent's performance during evaluation.

2. **Run Evaluation Episodes**: For each evaluation episode (ranging from 1 to `num_episodes`), the function:
    - Starts a timer to measure the episode duration.
    - Resets the environment to an initial state and initializes the episode reward and step count.
    - Executes the agent's policy in the environment until the episode is done. At each step, it:
        - Predicts the action using the agent's policy (deterministic prediction).
        - Takes the action and observes the next state, reward, and whether the episode is done.
        - Adds the reward to the episode reward and updates the state.
        - Increments the step count and prints the current step's reward.
    - Stops the timer when the episode is done and calculates the episode duration.
    - Appends the episode reward, duration, and step count to their respective lists.
    - Prints the progress (current episode, reward, duration, and steps).

3. **Calculate Averages**: After completing all evaluation episodes, the function calculates the average reward, duration, and steps across all episodes.

4. **Return Results**: The function returns the average reward, duration, and steps, which can be used to evaluate the agent's performance in the given environment.

The `evaluate_agent` function provides a standardized way to evaluate a trained agent's performance and can be used to compare the performance of different agents, training algorithms, or hyperparameters.


In [ ]:
def evaluate_agent(agent, env, num_episodes):
    total_rewards = []
    episode_durations = []
    episode_steps = []

    for episode in range(num_episodes):
        start_time = time.time()
        state = env.reset()
        done = False
        episode_reward = 0
        step_count = 0

        while not done:
            action, _ = agent.predict(state, deterministic=True)
            next_state, reward, done, _ = env.step(action)
            episode_reward += reward
            state = next_state
            step_count += 1
            print(f"Episode {episode + 1}, Step {step_count}: Reward={reward}")


        end_time = time.time()
        episode_duration = end_time - start_time
        total_rewards.append(episode_reward)
        episode_durations.append(episode_duration)
        episode_steps.append(step_count)

        # Print progress
        print(f"Episode {episode + 1}/{num_episodes}: Reward={episode_reward}, Duration={episode_duration:.2f}s, Steps={step_count}")

    avg_reward = np.mean(total_rewards)
    avg_duration = np.mean(episode_durations)
    avg_steps = np.mean(episode_steps)

    return avg_reward, avg_duration, avg_steps

In [ ]:
train_and_test()

In [ ]:
recommend()

##  Image Properties Preprocessing Code

The given code snippet performs the following tasks:

1. **Load Raw Image Properties**: It reads the raw image properties from the CSV file `./raw_image_properties.csv`, using the separator `|`, and stores the data in a pandas DataFrame `df`.

2. **Clean Image Properties**: The `clean` function is called on the raw image properties DataFrame `df` to clean the data. The cleaned data is stored in the DataFrame `clean_image_properties`.

3. **Find Maximum Width and Height**: The maximum width and height values from the cleaned image properties are calculated and stored in the variables `max_width` and `max_height`, respectively.

4. **Preprocess Image Properties**: The `preprocess` function is called on the cleaned image properties to perform feature engineering and scaling. The preprocessed data is stored in the DataFrame `processed_image_properties`.

5. **Save Processed Image Properties**: The preprocessed image properties are saved to a CSV file `./processed_image_properties.csv`, using the separator `|` and without including the DataFrame index.

6. **Extract Image Filenames**: The image filenames are extracted from the processed image properties and stored in a list called `images_filenames`.

7. **Create User Preferences**: A dictionary called `preferences` is created to store user preferences, including the camera make, tags, hex color, width, and height.

8. **Convert Preferences to DataFrame**: The user preferences dictionary is converted into a pandas DataFrame `df`, containing a single row with the preferences.

9. **Clean User Preferences**: The `clean_prefs` function is called on the user preferences DataFrame to clean the data. The cleaned data is stored in the DataFrame `clean_preferences`.

10. **Preprocess User Preferences**: The `preprocess_preferences` function is called on the cleaned user preferences DataFrame, along with the maximum width and height values and the column names of the processed image properties DataFrame. The preprocessed user preferences are stored in the DataFrame `prepro_preferences`.

This code snippet demonstrates how to preprocess both image properties and user preferences, which can be used later for model training, evaluation, or recommendation purposes.


In [ ]:
df = pd.read_csv('./raw_image_properties.csv', sep="|")
# Convert the list of ImageProperties objects to a pandas DataFrame
#df = pd.DataFrame(raw_image_properties)

clean_image_properties = clean(df)
max_width = clean_image_properties["width"].max()
max_height = clean_image_properties["height"].max()
processed_image_properties = preprocess(clean_image_properties)

# Save processed image properties to a CSV file
processed_image_properties.to_csv('./processed_image_properties.csv', sep="|", index=False)

images_filenames = processed_image_properties["name"].values.tolist()

tags = "['dog']"

preferences = {
    'make': "Nikon",
    'tags': tags,
    'hex_color': "#883131",
    'width': 2000,
    'height': 3000
}

df = pd.DataFrame([preferences])

clean_preferences = clean_prefs(df)

prepro_preferences = preprocess_preferences(clean_preferences, max_width, max_height, processed_image_properties.columns)


## Explanation of the Image Similarity Code

The given code snippet performs the following tasks:

1. **Create Feature Matrix**: The code creates a feature matrix by dropping the `name` column from the `processed_image_properties` DataFrame. This matrix contains only the feature values of the processed image properties.

2. **Replace NaN Values**: All the NaN values in the feature matrix are replaced with 0 using the `fillna()` method.

3. **Define Similarity Function**: A function named `get_top_n_similar_names_for_given_vector()` is defined, which takes three parameters: a given feature vector, the feature matrix, and an optional `top_n` parameter (with a default value of 3). This function computes the cosine similarity scores between the given vector and the feature matrix, sorts the scores in descending order, and returns the top `n` image names based on the similarity scores.

4. **Reorganize Preprocessed User Preferences**: The `prepro_preferences` DataFrame is reorganized to match the column order of the feature matrix. This step is necessary to ensure that the given vector and the feature matrix have the same feature order when computing similarity scores.

5. **Get Top Similar Images**: The given vector is obtained from the `prepro_preferences` DataFrame, and the `get_top_n_similar_names_for_given_vector()` function is called with the given vector and the feature matrix. The top 3 most similar image names are stored in the list `recommended_names`.

6. **Print Recommended Image Names**: The recommended image names are printed as the output.

This code snippet demonstrates how to find the top similar images for a given feature vector based on the cosine similarity metric. The top similar images can be used for recommendation purposes.

In [ ]:
feature_matrix = processed_image_properties.drop(columns=["name"], axis=1)
# replace all NaN values with 0
feature_matrix = feature_matrix.fillna(0)
# Function to get top_n most similar names for a given vector
def get_top_n_similar_names_for_given_vector(vector, feature_matrix, top_n=3):
    vector = vector.reshape(1, -1)
    similarity_scores = cosine_similarity(vector, feature_matrix)
    sorted_indexes = similarity_scores[0].argsort()[::-1]
    top_n_indexes = sorted_indexes[:top_n]  # Get the top_n indices
    return processed_image_properties['name'].iloc[top_n_indexes].tolist()


# reorganize the columns of prepo_preferences to match the order of feature_matrix
prepro_preferences = prepro_preferences[feature_matrix.columns]
# Example usage: Get the top 3 most similar names for a given vector
given_vector = prepro_preferences.values

recommended_names = get_top_n_similar_names_for_given_vector(given_vector, feature_matrix)
print(recommended_names)